# Reconocimiento de celebridades con Amazon Rekognition

Este notebook ofrece un walkthrough por la API de reconocimiento de celebridades en Amazon Rekognition. Puede identificar rápidamente a personas conocidas en nuestras bibliotecas de videos e imágenes para catalogar secuencias y fotos para casos de uso en la industria del marketing, la publicidad y los medios de comunicación.

    https://docs.aws.amazon.com/rekognition/latest/dg/celebrities.html
    https://docs.aws.amazon.com/rekognition/latest/dg/API_RecognizeCelebrities.html
    https://docs.aws.amazon.com/rekognition/latest/dg/API_StartCelebrityRecognition.html
    https://docs.aws.amazon.com/rekognition/latest/dg/API_GetCelebrityRecognition.html

## Check setup

In [ ]:
%store -r setup_dependencies_passed

In [ ]:
try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:


print(setup_dependencies_passed)



In [ ]:
if not setup_dependencies_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

## Setup notebook

In [ ]:
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os
import sagemaker
import boto3

In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name
rekognition = boto3.client("rekognition")
s3 = boto3.client("s3")

## Obtenemos la imagen y la subimos a S3

In [ ]:
!mkdir -p ./tmp
!curl -s -o ./tmp/gerard-pique.png "https://www.lecturas.com/medio/2022/11/06/gerard-pique-llorando_611ca467_1280x720.jpg"

In [ ]:
temp_folder = "tmp/"
image_name = "gerard-pique.png"

In [ ]:
s3.upload_file(
    f"./tmp/{image_name}",
    bucket,
    image_name
)

In [ ]:
display(IImage(url=s3.generate_presigned_url("get_object", Params={"Bucket": bucket, "Key": image_name})))

## Llamar a Amazon Rekognition para reconocer a los famosos en la imagen

https://docs.aws.amazon.com/rekognition/latest/dg/API_RecognizeCelebrities.html

In [ ]:
recognizeCelebritiesResponse = rekognition.recognize_celebrities(
    Image={
        "S3Object": {
            "Bucket": bucket,
            "Name": image_name,
        }
    }
)

In [ ]:
def drawBoundingBoxes(sourceImage, boxes):
    # blue, green, red, grey
    colors = ((255, 255, 255), (255, 255, 255), (76, 182, 252), (52, 194, 123))

    # Download image locally
    imageLocation = temp_folder + os.path.basename(sourceImage)
    s3.download_file(bucket, sourceImage, imageLocation)

    # Draws BB on Image
    bbImage = Image.open(imageLocation)
    draw = ImageDraw.Draw(bbImage)
    width, height = bbImage.size
    col = 0
    maxcol = len(colors)
    line = 3
    for box in boxes:
        x1 = int(box[1]["Left"] * width)
        y1 = int(box[1]["Top"] * height)
        x2 = int(box[1]["Left"] * width + box[1]["Width"] * width)
        y2 = int(box[1]["Top"] * height + box[1]["Height"] * height)

        draw.text((x1, y1), box[0], colors[col])
        for l in range(line):
            draw.rectangle((x1 - l, y1 - l, x2 + l, y2 + l), outline=colors[col])
        col = (col + 1) % maxcol

    imageFormat = "PNG"
    ext = sourceImage.lower()
    if ext.endswith("jpg") or ext.endswith("jpeg"):
        imageFormat = "JPEG"

    bbImage.save(imageLocation, format=imageFormat)

    display(bbImage)

## Revisar la respuesta JSON devuelta por la API de reconocimiento de celebridades de Rekognition

In [ ]:
display(recognizeCelebritiesResponse)

## Mostrar imagen con cuadros delimitadores alrededor de las celebridades reconocidas

In [ ]:
boxes = []
celebrities = recognizeCelebritiesResponse["CelebrityFaces"]
for celebrity in celebrities:
    boxes.append((celebrity["Name"], celebrity["Face"]["BoundingBox"]))

drawBoundingBoxes(image_name, boxes)

## Liberar recursos

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.b>p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernelbutton>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
script>
